In [1]:
%pylab inline
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
from multiprocessing import Pool
from functools import partial 
import warnings
warnings.filterwarnings("ignore")
np.printoptions(precision=2)

# time line
T_min = 20
T_max = 80
T_y = 65
beta = 1/(1+0.02)
# States of the economy, GOOD or BAD, {1 : GOOD}, {0 : BAD}
S = [0,1]
# All the money amount are denoted in thousand dollars
earningShock = [0.8,1.2]
# Define transition matrix of economical states
# GOOD -> GOOD 0.8, BAD -> BAD 0.6 
P = np.array([[0.6, 0.4],[0.2, 0.8]])
# current risk free interest rate 
r_f = [0.01 ,0.03]

# stock return depends on current and future econ states
def r_m(current_state, next_state):
    if (current_state == 1 and next_state == 1):
        return 0.20
    elif (current_state == 1 and next_state == 0):
        return -0.15
    elif (current_state == 0 and next_state == 1):
        return 0.15
    else:
        return -0.20
    
#Define the utility function
def u(c):
    return np.log(c)
        

# Define deterministic function to captures the hump shape of life-cycle earnings.
ageTick = [20, 50, 60, 65, 80]
earning = [35, 100, 70, 35, 35]
linearFit = interp1d(ageTick, earning, kind = "linear")
detEarning = np.array(linearFit(np.linspace(T_min, T_max, T_max - T_min + 1)))

def y(age, s):
    if age <= T_y:
        return detEarning[age-T_min-1] * earningShock[s]
    else:
        return detEarning[age-T_min-1]

# Define the transtiion of state 
def transition(Z, a):
    '''
        Input: Z is a tuple containing states variables: (w, s) which is the current wealth and econ state
               a is the action taken with contains variables: (c, b, k) which is the combination of consumption, bond and stock
        Output: the next possible states with corresponding probabilities
    '''
    w, s = Z
    c, b, k = a
    # collect possible next state (w_next, s_next) with probability prob
    Z_next = []
    prob = []
    # depend on the current econ state s and future state s_next we have the following return on bond and stock 
    for s_next in S:
        w_next = b*(1+r_f[s]) + k*(1+r_m(s,s_next))
        Z_next.append([w_next, s_next])
        prob.append(P[s, s_next])
    return Z_next, prob

Populating the interactive namespace from numpy and matplotlib


In [2]:
# timeline is from age 15 - 80, use approximation for value function from age 15 - 79 by discretizing the state space
# since we know the terminal condition. Here XX is the grid point collection. Initiate the grid and corresponding value.

# This function is used to generate power grid 
# def powspace(start, stop, power, num):
#     start = np.power(start, 1/float(power))
#     stop = np.power(stop, 1/float(power))
#     return np.power(np.linspace(start, stop, num=num), power) 

w_grid_size = 1000
w_lower = 0
w_upper = 1200
T = T_max - T_min
# wgrid = powspace(w_lower, w_upper, 3, num = w_grid_size)
wgrid = np.linspace(w_lower, w_upper, num = w_grid_size)

print("Shape of grid w grid: ", wgrid.shape)
# V use to store the value 
V_t = np.zeros((w_grid_size,len(S),T))
C_t = np.zeros((w_grid_size,len(S),T))
B_t = np.zeros((w_grid_size,len(S),T))
K_t = np.zeros((w_grid_size,len(S),T))
print("Shape of Value matrix: ", V_t.shape)

# Calculate terminal value function 
V_t[:, 0, T-1] = u(y(T_max,0) + wgrid)
V_t[:, 1, T-1] = u(y(T_max,1) + wgrid)
C_t[:, 0, T-1] = y(T_max,0) + wgrid
C_t[:, 1, T-1] = y(T_max,1) + wgrid
B_t[:, 0, T-1] = 0
B_t[:, 1, T-1] = 0
K_t[:, 0, T-1] = 0
K_t[:, 1, T-1] = 0

Shape of grid w grid:  (1000,)
Shape of Value matrix:  (1000, 2, 60)


In [3]:
from scipy.optimize import minimize, minimize_scalar
# Definition of the value function, return of the funciton is the value of the function and the coresponding policy
theta_grid_size = 101
theta_grid = np.linspace(start = 0, stop = 1, num = 101)
vals = np.zeros(theta_grid_size)
bks = np.zeros(theta_grid_size)

def V(w, s, t, model):
    def obj(bk, theta):
        b = bk * theta
        k = bk * (1 - theta)
        c = y(t,s) + w - bk
        if c <= 0 or bk <= 0:
            return 9999999999999999 
        Z_next, prob = transition([w,s], (c, b, k))
        V_next = np.array([model[z[1]](z[0]) for z in Z_next])
        return -(u(c) + beta * np.dot(V_next, prob))
    
    def f(the):
        res = minimize_scalar(obj, args = (the))
        return [-res.fun, res.x]
    results = np.array(list(map(f, theta_grid)))
    vals = results[:,0]
    bks = results[:,1]                    
    index = np.argmax(vals)
    max_val = vals[index]
    max_the = theta_grid[index]
    max_bk = bks[index]
    return np.array([max_val, [max_bk*max_the, max_bk*(1-max_the)]])

In [ ]:
pool = Pool()
# Set value to the grid by backward induction 
for t in range(T_max - 1, T_min - 1, -1):
    print(t)
    tt = t-T_min-1
    cs = [interp1d(wgrid, V_t[:,0,tt+1], kind = "cubic", fill_value="extrapolate"),
          interp1d(wgrid, V_t[:,1,tt+1], kind = "cubic", fill_value="extrapolate")]    
    for s in S:                                               
        f = partial(V, s = s, t = t, model = cs)
        results = np.array(pool.map(f, wgrid))
        V_t[:,s,tt] = results[:,0]
#        B_t[:,s,tt] = np.array([b for b in results[:,1]])
        B_t[:,s,tt] = np.array([b[0] for b in results[:,1]])
        K_t[:,s,tt] = np.array([k[1] for k in results[:,1]])
        C_t[:,s,tt] = wgrid + y(t,s) - B_t[:,s,tt] - K_t[:,s,tt]
pool.close()

79
78
77
76
75
74
73
72
71
70
69
68
67
66
65
64
63
62
